In [60]:
import cv2
import numpy as np

In [61]:
def resize_and_crop_image(image):
    ratio = 1
    max_edge = max(image.shape[0], image.shape[1])
    while max_edge > 1000:
        ratio *= 2
        max_edge //= 2

    rows = image.shape[0] // ratio
    cols = image.shape[1] // ratio

    # print(f"resize ratio = {ratio}.")
    image_resized = cv2.resize(image, (cols, rows))
    # print(image_resized.shape)
    if CROP:
        size = min(cols, rows)
        return image_resized[:size, :size]
    else:
        return image_resized

In [62]:
def RGBRange(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_skin = np.array([0, 48, 80], dtype="uint8")
    upper_skin = np.array([20, 255, 255], dtype="uint8")
    skinMask = cv2.inRange(hsv, lower_skin, upper_skin)
    skin = cv2.bitwise_and(image, image, mask=skinMask)
    return skin

In [63]:
def morph_image(image,morph, kernel_size):
    def erosion(image_cv, kernel_size = 4):
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_size,kernel_size))
        image_cv_eroded = cv2.erode(image_cv, kernel)
        return image_cv_eroded
    def dilate(image_cv, kernel_size = 4):
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_size,kernel_size))
        image_cv_dilated = cv2.dilate(image_cv, kernel)
        return image_cv_dilated
    if morph == 'dilate':
        image_morphed = dilate(image,kernel_size = kernel_size)
    elif morph == 'erode':
        image_morphed = erosion(image,kernel_size = kernel_size)
    else:
        raise NotImplementedError
    return image_morphed

def BinaryRange(image):
    threshold = 145
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, img_thresh = cv2.threshold(img_gray, threshold, 255, cv2.THRESH_BINARY_INV)
    img_thresh = morph_image(img_thresh,morph='erode',kernel_size=6)
    img_thresh = morph_image(img_thresh,morph='dilate',kernel_size=6)
    return img_thresh

In [64]:
EXP = True
CROP = False

In [65]:
mode = "gray"
process_method = None
if mode == "rgb":
    process_method = RGBRange
elif mode == "gray":
    process_method = BinaryRange
    
assert process_method is not None

def process_func(image):
    return process_method(resize_and_crop_image(image))


In [66]:
import os

# Define paths
VIDEO_NAME = "fingers"
FVideo = f'videos/{VIDEO_NAME}.mp4'
WORKDIR = f"videos/{VIDEO_NAME}/"
os.makedirs(WORKDIR, exist_ok=True)
FFirst = WORKDIR + "first.png"  # Filename for the first frame image
FOutput = WORKDIR + f"{mode}.mp4"  # Filename for the output video

In [67]:
# Create video capture object
vidCap = cv2.VideoCapture(FVideo)

# Get video properties for the output video

fps = vidCap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    

# Process video frames
success, image = vidCap.read()
assert success

cv2.imwrite(FFirst, image)

skin = process_func(image)
# Show the result
cv2.imshow("Skin detection", skin)
cv2.waitKey(0)
cv2.destroyAllWindows()
        
if not EXP:
    # frame_width = int(vidCap.get(cv2.CAP_PROP_FRAME_WIDTH))
    # frame_height = int(vidCap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_width = skin.shape[1]
    frame_height = skin.shape[0]
    out = cv2.VideoWriter(FOutput, fourcc, fps, (frame_width, frame_height))
    
    while success:
        # Apply skin detection
        skin = process_func(image)
        # Write the processed frame
        out.write(skin)
        success, image = vidCap.read()

    out.release()
    

# Release resources
vidCap.release()
cv2.destroyAllWindows()
